In [2]:
# load Korpora
import collections
from Korpora import Korpora
kowikitext = Korpora.load('kowikitext')

print(len(kowikitext.train))
print(len(kowikitext.dev))
print(len(kowikitext.test))



    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/kowikitext
    References :

    한국어 위키피디아의 덤프 데이터를 바탕을 제작한 wikitext 형식의 텍스트 파일입니다.
    학습 및 평가를 위하여 위키페이지 별로 train (99%), dev (0.5%), test (0.5%) 로 나뉘어져있습니다.


    # License
    CC-BY-SA 3.0 which kowiki dump dataset is licensed

[Korpora] Corpus `kowikitext` is already installed at C:\Users\USER\Korpora\kowikitext\kowikitext_20200920.train.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\USER\Korpora\kowikitext\kowikitext_20200920.train
[Korpora] Corpus `kowikitext` is already installed at C:\Users\USER\Korpora\kowikitext\kowikitext_20200920.test.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\USER\Korpora\kowi

In [18]:
# Char Tokenizer
char_counter = collections.defaultdict(int)

for i, line in enumerate(kowikitext.train):
    line = line.text.strip()

    for c in line:
        char_counter[c] += 1

print(len(char_counter))

most_freq = sorted(char_counter.items(), key=lambda item: item[1], reverse=True)
print(most_freq[:10])

least_freq = sorted(char_counter.items(), key=lambda item: item[1])
print(least_freq[:10])

4823
[(' ', 601985), ('이', 54291), ('.', 45556), ('다', 43797), ('1', 42661), ('\n', 38995), ('에', 37239), ('의', 37110), (',', 34664), ('0', 34390)]
[('朋', 1), ('쑨', 1), ('孙', 1), ('鉛', 1), ('铜', 1), ('扫', 1), ('嬿', 1), ('扬', 1), ('璇', 1), ('晨', 1)]


In [21]:
char_to_id = {'[PAD]': 0, '[UNK]': 1}
for c, cnt in char_counter.items():
    char_to_id[c] = len(char_to_id)
print(len(char_to_id))

4825


In [ ]:
for i, line in enumerate(kowikitext.train):
    if i >= 5:
        break
    _ids = [char_to_id[c] for c in line.text]
    print(line.text)
    print(_ids)

- Char Tokenizer 장점
  - 모든 문장을 적은 수의 vocab으로 표현 가능
  - OOV(Out Of Vocabulary) 문제가 발생할 가능성이 낮음
- Char Tokenizer 단점
  - token 수가 많아지므로 연산량이 많고, 학습속도가 늦다
  - 단어의 의미를 표현하지 못할 가능성이 높다. (이를 해결하기 위해 layer를 많이 쌓는다)

In [3]:
# Word Tokenizer
word_counter = collections.defaultdict(int)

for i, line in enumerate(kowikitext.train):
    line = line.text.strip()
    for w in line.split():
        word_counter[w] += 1

print(len(word_counter))

most_freq = sorted(word_counter.items(), key=lambda item: item[1], reverse=True)
print(most_freq[:10])

least_freq = sorted(word_counter.items(), key=lambda item: item[1])
print(least_freq[:10])

176098
[('-', 4883), (':', 3274), ('있다.', 2793), ('!', 2628), ('이', 2240), ('수', 2122), ('역', 1685), ('~', 1647), ('그', 1539), ('*', 1527)]
[('김세권(1931년', 1), (',金世權)', 1), ('검사장을', 1), ('서울시에서', 1), ('경기중학교,1982년', 1), ('12일자', 1), ('서울고등학교,1981년', 1), ('사법과에서', 1), ('합격하였다.', 1), ('김세권은', 1)]


In [20]:
word_to_id = {'[PAD]': 0, '[UNK]': 1}
for w, cnt in word_counter.items():
    word_to_id[w] = len(word_to_id)
print(len(word_to_id))

176100


In [ ]:
for i, line in enumerate(kowikitext.train):
    if i >= 5:
        break
    _ids = [word_to_id[w] for w in line.text.strip().split()]
    print(line.text)
    print(_ids)

- Word Tokenizer 장점
  - token의 수가 적다
- Word Tokenizer 단점
  - 활용형으로 인해 유사 단어들이 많아지고, 각각이 다른 의미를 가질 수 있다.
  - Vocab 수가 매우 많아지므로, 메모리 사용량과 연산량이 증가한다.

- Morph Tokenizer 장점
  - 형태소 단위로 분할하기 때문에, 각 token이 적당한 의미를 가짐
  - char tokenizer와 word tokenizer의 중간 정도의 token 갯수를 가짐
- Morph Tokenizer 단점
  - 발전 속도가 매우 늦고, 형태소 분석기들이 어느정도 오류가 있음
  - Vocab 수가 많음

In [6]:
# BPE (Byte Pair Encoding)
bpe_counter = collections.defaultdict(int)
c = 0
for w, n in word_counter.items():
    c += 1
    if c > 5:
        break
    print(w, n)
    w = f"\u2581{w}"
    bpe_counter[" ".join(w)] = n

print(bpe_counter)

외교부장 2
공원 26
김세권(1931년 1
~ 1647
,金世權) 1
defaultdict(<class 'int'>, {'▁ 외 교 부 장': 2, '▁ 공 원': 26, '▁ 김 세 권 ( 1 9 3 1 년': 1, '▁ ~': 1647, '▁ , 金 世 權 )': 1})


In [9]:
import re

def update_vocab(vocab, counter):
    for w in counter:
        for s in w.split():
            if s not in vocab:
                vocab[s] = len(vocab)

    return vocab

# bi-gram 빈도 계산
def get_stats(counter):
    pairs = collections.defaultdict(int)
    for word, freq in counter.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq

    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [8]:
bpe_to_id = {'[PAD]': 0, '[UNK]': 1}
bpe_to_id = update_vocab(bpe_to_id, bpe_counter)
print(bpe_to_id)

{'[PAD]': 0, '[UNK]': 1, '▁': 2, '외': 3, '교': 4, '부': 5, '장': 6, '공': 7, '원': 8, '김': 9, '세': 10, '권': 11, '(': 12, '1': 13, '9': 14, '3': 15, '년': 16, '~': 17, ',': 18, '金': 19, '世': 20, '權': 21, ')': 22}


In [18]:
# 아래 과정 반복


pairs = get_stats(bpe_counter)
print(pairs)

best = max(pairs, key=pairs.get)
print(best)

bpe_counter = merge_vocab(best, bpe_counter)
print(bpe_counter)

bpe_to_id = update_vocab(bpe_to_id, bpe_counter)
print(bpe_to_id)

defaultdict(<class 'int'>, {('▁외교부', '장'): 2, ('▁', '김'): 1, ('김', '세'): 1, ('세', '권'): 1, ('권', '('): 1, ('(', '1'): 1, ('1', '9'): 1, ('9', '3'): 1, ('3', '1'): 1, ('1', '년'): 1, ('▁', ','): 1, (',', '金'): 1, ('金', '世'): 1, ('世', '權'): 1, ('權', ')'): 1})
('▁외교부', '장')
{'▁외교부장': 2, '▁공원': 26, '▁ 김 세 권 ( 1 9 3 1 년': 1, '▁~': 1647, '▁ , 金 世 權 )': 1}
{'[PAD]': 0, '[UNK]': 1, '▁': 2, '외': 3, '교': 4, '부': 5, '장': 6, '공': 7, '원': 8, '김': 9, '세': 10, '권': 11, '(': 12, '1': 13, '9': 14, '3': 15, '년': 16, '~': 17, ',': 18, '金': 19, '世': 20, '權': 21, ')': 22, '▁공': 23, '▁~': 24, '▁공원': 25, '▁외': 26, '▁외교': 27, '▁외교부': 28, '▁외교부장': 29}


- BPE Tokenizer 장점
  - 말뭉치가 있으면 비교적 간단하게 만들 수 있음
  - subword를 이용하면 적은 수의 vocab으로 OOV를 최소화 할 수 있음
- BPE Tokenizer 단점
  - subword의 분할이 의미기준이 아닐 수 있음

In [21]:
import shutil
import sentencepiece as spm

def train_sentencepiece(corpus, prefix, vocab_size=32000):
    spm.SentencePieceTrainer.Train(
        f"--input={corpus}" +
        f" --model_prefix={prefix}" +
        f" --vocab_size={vocab_size + 7}" +  # 7은 특수토큰 개수
        " --model_type=unigram" +
        " --max_sentence_length=999999" +  # 문장 최대 길이
        " --pad_id=0 --pad_piece=[PAD]" +  # pad token 및 id 지정
        " --unk_id=1 --unk_piece=[UNK]" +  # unknown token 및 id 지정
        " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence token 및 id 지정
        " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence token 및 id 지정
        " --user_defined_symbols=[SEP],[CLS],[MASK]" +  # 기타 추가 토큰 SEP: 4, CLS: 5, MASK: 6
        " --input_sentence_size=100000" +  # 말뭉치에서 셈플링해서 학습
        " --shuffle_input_sentence=true")  # 셈플링한 말뭉치 shuffle

corpus_dir = "C:/Users/USER/Korpora/kowikitext/"

train_sentencepiece(corpus_dir + "kowikitext_20200920.train", "kowiki_32000")

shutil.copy("kowiki_32000.model", corpus_dir)
shutil.copy("kowiki_32000.vocab", corpus_dir)

In [23]:
spm_vocab = spm.SentencePieceProcessor()
spm_vocab.load(corpus_dir + "kowiki_32000.model")

for i, line in enumerate(kowikitext.train):
    if i >= 5:
        break

    line = line.text.strip()
    print(line)

    tokens = spm_vocab.EncodeAsPieces(line)
    print(tokens)

    _ids = spm_vocab.EncodeAsIds(line)
    print(_ids)

외교부장
외교부장
['▁외교', '부장', '▁외교', '부장']
[4042, 5440, 4042, 5440]
공원
공원
['▁공원', '▁공원']
[3619, 3619]
김세권(1931년 ~ ,金世權) 은 제16대 서울고등검찰청 검사장을 역임한 법조인이다.
['▁김세', '권', '(', '19', '31', '년', '▁~', '▁', ',', '金', '世', '權', ')', '▁', '은', '▁제', '16', '대', '▁서울', '고등검찰청', '▁검사', '장을', '▁역임', '한', '▁법조인', '이다', '.']
[9521, 308, 15, 710, 1334, 14, 79, 9, 10, 984, 3136, 5385, 12, 9, 20, 49, 936, 81, 509, 25041, 1977, 2293, 3331, 31, 17099, 38, 8]
1931년 서울시에서 태어나 경기중학교,1982년 4월 12일자 매일경제 서울고등학교,1981년 4월 25일자 동아일보 1956년 서울대학교 법학과를 나온 후 1956년 제8회 고등고시 사법과에서 합격하였다. 1958년 서울지방검찰청 검사에 임용되었다.
김세권은 두산그룹 창업주인 박두병 딸인 박용언과 결혼했다.  김세권과 박용언은 아들은 1970년대 봉제업으로 성장한 태흥의 창업주 권태흥의 딸 권혜경과 결혼한 김형일 일경산업개발 부회장으로 1990년대 초반 대한민국에 게스·폴로 등을 수입해 유명세를 탔던 기업가다. 딸 김희정의 남편은 최원현 케이씨엘 대표변호사다.박승직 박두병 박용곤 박정원 재벌가 4대 33명 결혼 스토리
대검찰청 차장으로 재직하던 1986년 2월 전국 검사장 회의를 주재하면서 "국법질서 확립을 위한 검찰의 과제"라는 주제로 토의를 하여 국법질서와 사회기강 확립, 경제도약의 전기가 될 수 있는 현재의 국내외 경제여건을 최대한 유지, 활용할 수 있도록 경제질서교란사범 엄단, 민생을 불안하게 하는 반윤리적인 강력사범에 대한 단호한 응징, 사회적 신분과 지위의 고하를 가리지 않는 엄정 공

: 